# Train데이터 불러오기

각 년도별 DataFrame을 리스트에 append합니다.

리스트 인덱스별로

0: 2017년도 데이터   

1: 2018년도 데이터 

2: 2019년도 데이터 

3: 202년도 데이터 

입니다

In [1]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import warnings

warnings.filterwarnings(action='ignore') 
know_train = [pd.read_csv(path) for path in sorted(glob('KNOW_*.csv'))]

In [77]:
know_train[1].iloc[0:10,125:] # 2017년도 샘플

,bq32,bq33,bq34,bq35,bq36,bq37,bq37_1,bq38,bq38_1,bq38_2,bq39,bq40,bq41_1,bq41_2,bq41_3,knowcode
0,403,1,6,2,18,2,1551,1,1,0,1,62,157,114,0,306301
1,1641,412,6,2,20,1,1206,1,2,0,1,57,73,41,0,133204
2,1,1,8,2,87,1,303,3,0,6,0,36,0,0,45,702502
3,1,1,4,2,42,3,522,1,1,0,1,57,134,89,0,121201
4,1,1,8,4,18,3,1552,1,1,0,1,57,86,70,0,306301
5,1143,412,6,4,16,3,56,1,1,0,1,62,86,55,0,133204
6,1,1,6,2,32,4,337,1,1,0,1,57,127,91,0,121201
7,1,1,6,4,18,3,1553,1,2,0,1,54,108,56,0,306401
8,1,1,6,2,40,2,1550,1,1,0,1,62,165,55,0,306301
9,1752,417,8,2,40,5,759,1,2,0,2,62,176,119,0,121201


# 전처리 

데이터의 빈 셀에 None값이 아닌 ' '처럼 공백으로 들어가있기 때문에 is_null등의 함수로 결측치를 찾아낼 수 없습니다. 

공백이 있는 컬럼은 '0'으로 대체하였습니다.

In [3]:
for df in know_train:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

## 라벨 인코딩

숫자로 변환할 수 있는 컬럼은 라벨 인코딩을 사용하지 않았습니다.

string이나 object컬럼은 라벨인코더를 이용해 변환하였으며 추후 test셋에 사용해야하기 때문에 년도별, 컬럼별로 dictionary를 이용해 저장하였습니다

In [4]:
from sklearn.preprocessing import LabelEncoder
years = ['2017', '2018', '2019', '2020']

year_encoder = {}

for year, df in zip(years, know_train):
    print(year)
    encoders = {}
    
    for col in df.columns:
        if col == 'ID':
            continue
        
        try:
            df[col] = df[col].map(int)
        except:
            encoder = LabelEncoder()
            df[col] = df[col].map(str)
            df[col] = encoder.fit_transform(df[col])
            encoders[col] = encoder
            
            
    year_encoder[year] = encoders

2017
2018
2019
2020


# X, y 구분 및 모델 학습

이번 대회에서 맞춰야 할 값은 knowcode입니다.

ID와 knowcode를 제외한 나머지 feature를 X, knowcode를 정답 y로 두어 모델을 학습하였습니다.

베이스라인에서는 의사결정나무와 랜덤포레스트를 선정하였습니다

In [5]:
train_data = {}
for year, df in zip(years, know_train):
    train_data[year] = {'X': df.iloc[:, 1:-1], # ID제외
                        'y': df.iloc[:, -1]} 

## 의사결정나무

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

dt_models = {}

for year in tqdm(years):
    model = DecisionTreeClassifier(random_state=123456)
    model.fit(train_data[year]['X'].iloc[:, :], train_data[year]['y'])
    dt_models[year] = model

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:10<00:00,  2.51s/it]


## 랜덤포레스트

In [32]:

rf_models = {}

for year in tqdm(years):
    model = RandomForestClassifier(n_estimators=100, random_state=123456, n_jobs=8)
    model.fit(train_data[year]['X'].iloc[:, :], train_data[year]['y'])
    rf_models[year] = model

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:54<00:00, 13.75s/it]


### xgboost

In [43]:
xgb_models = []
xgb_models.append(model)

In [44]:
from xgboost import XGBClassifier

xgb_models = []

for year in tqdm(years):
    model = XGBClassifier()
    model.fit(train_data[year]['X'].iloc[:, :], train_data[year]['y'])
    xgb_models.append(model)

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

[22:55:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 25%|████████████████████▊                                                              | 1/4 [08:34<25:44, 514.93s/it]

[23:04:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 50%|█████████████████████████████████████████▌                                         | 2/4 [16:29<16:22, 491.35s/it]

[23:12:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 75%|██████████████████████████████████████████████████████████████▎                    | 3/4 [40:29<15:24, 924.58s/it]

[23:36:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|████████████████████████████████████████████████████████████████████████████████| 4/4 [1:09:35<00:00, 1043.76s/it]


### LGBM

In [54]:
from lightgbm import LGBMClassifier

lgb_models = []

for year in tqdm(years):
    model = LGBMClassifier(n_estimators=100)
    model.fit(train_data[year]['X'].iloc[:, :], train_data[year]['y'])
    lgb_models.append(model)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [04:17<00:00, 64.50s/it]


## Testset 불러오기

마찬가지로 년도별로 DataFrame으로 불러온 후 리스트에 할당합니다.


In [17]:
know_test = [pd.read_csv(path) for path in sorted(glob('KNOW_*_test.csv'))]
know_test[0].head() # 2017년도 test 샘플

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,...,bq36,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3
0,0,3,4,2,2,3,3,1,,3,...,2,26,3,비서학,1,1,1,3000,,2300
1,1,5,5,3,5,5,5,5,5,4,...,1,57,4,농화학,1,1,1,5500,,2500
2,2,5,5,5,4,5,4,1,,1,...,1,31,4,신문방송,1,1,1,4300,,4000
3,3,4,5,5,6,4,6,3,4,4,...,1,35,6,화학,1,1,1,4100,,3000
4,4,5,6,4,5,4,5,1,,1,...,1,36,4,광고홍보,1,1,1,2800,,2000


Train set과 마찬가지로 평균, '0'으로 대체하였습니다

In [18]:
for df in know_test:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

Train에 사용하였던 라벨인코더를 이용해서 카테고리를 정수로 변환합니다.

만약 Test변수가 Train 라벨인코더의 클래스에 없을경우 -1로 처리합니다

In [19]:
years = ['2017', '2018', '2019', '2020']

for year, df in zip(years, know_test):
    print(year)
    encoders = {}
    
    for col in df.columns:
        
        try:
            df[col] = df[col].map(int)
        except:
            encoder = year_encoder[year][col]
            df[col] = df[col].map(str)
            category_map = {category: idx for idx, category in enumerate(encoder.classes_)}
            df[col] = df[col].apply(lambda x: category_map[x] if x in category_map else -1) # train set에서 보지못한 카테고리변수 -1(UNK) 처리
            
            

2017
2018
2019
2020


### 데이터 처리 및 라벨인코딩 후 

In [20]:
know_train[0]['aq1_1'].value_counts()

3    3116
4    3007
2    1706
5    1072
1     585
Name: aq1_1, dtype: int64

In [21]:
know_test[0].head() # 2017년도 test 샘플

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,...,bq36,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3
0,0,3,4,2,2,3,3,1,0,3,...,2,26,3,497,1,1,1,3000,0,2300
1,1,5,5,3,5,5,5,5,5,4,...,1,57,4,287,1,1,1,5500,0,2500
2,2,5,5,5,4,5,4,1,0,1,...,1,31,4,705,1,1,1,4300,0,4000
3,3,4,5,5,6,4,6,3,4,4,...,1,35,6,1423,1,1,1,4100,0,3000
4,4,5,6,4,5,4,5,1,0,1,...,1,36,4,141,1,1,1,2800,0,2000


## 테스트셋 추출 및 학습

ID 값을 제외한 나머지 데이터를 이용하여 모델에 넣어 추론합니다.

In [22]:
test_data = {}
for year, df in zip(years, know_test):
    test_data[year] =  {'X': df.iloc[:,1:]}

### 의사결정나무로 예측

In [23]:
dt_predicts = [] 

for year in tqdm(years):
    pred = dt_models[year].predict(test_data[year]['X'])
    dt_predicts.extend(pred)
    
    

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  5.93it/s]


### 랜덤포레스트로 예측

In [24]:
rf_predicts = [] 

for year in tqdm(years):
    pred = rf_models[year].predict(test_data[year]['X'])
    rf_predicts.extend(pred)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:28<00:00,  7.05s/it]


### XGBoost로 예측

In [45]:
xgb_predicts = []

for i,year in tqdm(enumerate(years)):
    pred = xgb_models[i].predict(test_data[year]['X'])
    xgb_predicts.extend(pred)

4it [00:03,  1.02it/s]


### LGBM로 예측

In [59]:
lgb_predicts = []

for i,year in tqdm(enumerate(years)):
    pred = lgb_models[i].predict(test_data[year]['X'])
    lgb_predicts.extend(pred)

4it [00:39,  9.88s/it]


# 제출

In [62]:
submission = pd.read_csv('sample_submission.csv') # sample submission 불러오기

In [63]:
submission['knowcode'] = lgb_predicts

submission.to_csv('submission.csv', index=False)